# Accessing IMERG-Late Data in the Cloud (Algeria)

Parts of northern Algeria and Tunisia experienced flash floods in May 2023. Let's use the IMERG-Late product to quantify the total precipitation that fell across the country on one day in that region.

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

## Using `earthaccess`

While IMERG-Late is produced in 30-minute intervals, today we'll be using a version of the data that have been aggregated to daily time steps.

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDL',
    temporal = ('2023-05-12', '2023-05-13'))
results[0]

In [ ]:
# NOTE: open() requires a sequence of file references
files = earthaccess.open(results)
files

In [ ]:
ds = xr.open_dataset(files[0])
ds

There are a lot of different variables in this dataset, all described in [the IMERG-Late documentation.](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGHHL_06/summary). We'll use `precipitationCal`, which is the gauge-calibrated, multi-satellite estimate that NASA recommends for general use.

In [ ]:
ds['precipitationCal']

In [ ]:
ds['precipitationCal'].plot()

That looks weird! Any ideas about what is wrong?

We need to rotate the plot so that the rows of the image correspond to latitude bands. **Specifically, we need to tell `xarray` that longitude ("lon") should span the X-axis and latitude ("lat") should span the Y-axis.**

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat')

This looks better. IMERG doesn't make estimates over ice and snow-covered area, so you may recognize parts of Iceland, Norway, and Sweden in the upper-center of the image.

Why is most of the image dark?

The easiest way to see more detail in the image is to tell `xarray` to stretch the colorbar so that extreme values don't dominate; we do this with `robust = True`.

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat', robust = True)

Alternatively, we could tell `xarray` what the maximum value assigned to a color should be, using the `vmax` keyword argument. There's a corresponding `vmin` argument for the minimum value.

In [ ]:
ds['precipitationCal'].plot(x = 'lon', y = 'lat', vmax = 15)

In [ ]:
ds['precipitationCal'].sel(lat = slice(30, 40), lon = slice(0, 6)).plot()

In [ ]:
precip = ds['precipitationCal'].sel(lat = slice(30, 40), lon = slice(0, 6))

## Using `cartopy`

In [ ]:
from cartopy import crs

style = {
    'projection': crs.Orthographic(3, 36),
    'facecolor': 'lightgray'
}

plot = precip.plot(subplot_kws = style, transform = crs.Orthographic(3, 36))
plot.axes.coastlines()